In [1]:
####################################
# Special code for Jupyter Notebook
####################################
# Show plots in the notebook

%matplotlib inline 
#%matplotlib notebook 

# Automatically reload any packages that changed
#%load_ext autoreload
#%autoreload 2

# Display multiple arguments
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

##################
# Import librarys
##################
# Third party libraries

import os
import glob
import pandas as pd
import numpy as np
import statistics as stat
import scipy as sp
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import OrcFxAPI


# Course library
# import pywasp_edu as pw

############################################
# Limit number of rows to print from Pandas
############################################
pd.options.display.max_rows = 40

pi = np.arccos(-1)

# Used in order to embed figures in the text
from IPython.display import Image  

# Data extraction of time histories from Orcaflex

### Assumes that all *.sim files are of similar structure

In [2]:
class TH(object):
    
    def __init__(self, OrcFxobject, variableName, period, objectExtra):
        self.OrcFxobject = OrcFxobject
        self.variableName = variableName
        self.period = period
        self.objectExtra = objectExtra
        self.THvalues = self.OrcFxobject.TimeHistory(self.variableName, self.period, self.objectExtra)
        
    def getTH(self):
        return self.THvalues

    def getTHmax(self):
        return max(self.THvalues)
    
    def getTHmin(self):
        return min(self.THvalues)
    
    def getTHavg(self):
        return sum(self.THvalues)/len(self.THvalues)
    
    def getTHabsmax(self):
        return max(abs(self.getTHmax()), abs(self.getTHmin()))
    
    def getTHstddev(self):
        return stat.stdev(self.THvalues)
    


In [3]:
# Function for data extraction

def ExtractTHData(model, extraction_def):
    
    object_name = extraction_def[0]
    variableName = extraction_def[1]
    objectExtra = extraction_def[2]
    
    extraction = TH(model[object_name] ,variableName, OrcFxAPI.SpecifiedPeriod(t_start_res, t_end_res), objectExtra)
    
    return extraction
    

In [4]:
# Define extractions and dataframe 
# General details
t_start_res = 0 # [s]
t_end_res = 600 # [s]

# Folder with files
folder = "D:/100 - Load case batch runs/02 - Batch Runs/07 - Steady state power curve fixed floater - UMain contr WE_mode2"

# Make list of *.sim files
sim_files = []

os.chdir(folder)
for file in glob.glob("*.sim"):
    sim_files.append(file)
    
cols = ['Sim-file', 'Average','Max','Min','Abs max','std. dev']

# Blade root moments and tower moments
object_names = ["15MW RWT","15MW RWT","tower","tower"]
OrcFxVariableNames = ["Root connection Ey moment", "Root connection Ex moment","x bend moment", "y bend moment"]
GenVariableNames = ["Blade Root Moment, Flapwise", "Blade Root Moment, Edgewise","Tower Base Moment, Side-Side", "Tower Base Moment, Fore-Aft"]
objectExtras = [OrcFxAPI.oeTurbine(1), OrcFxAPI.oeTurbine(1), OrcFxAPI.oeEndB, OrcFxAPI.oeEndB]

# Define dataframe for extracted values
base_df = pd.DataFrame(index=np.arange(len(sim_files)) ,columns=cols)


# define n dataframe corresponding to n variables
df_str = []
for i in range(len(GenVariableNames)):
    print('folder_10[' + GenVariableNames[i] + ']')
    df_str.append('folder_10[' + GenVariableNames[i] + ']')

df = df_str.copy()
for i in range(len(GenVariableNames)):
    df[i] = base_df.copy()
    
    
#for sim_file in sim_files:
for (idx, sim_file) in enumerate(sim_files, 1):
    
    print(sim_file)
    
    model = OrcFxAPI.Model(sim_file)
    
    # Loop through parameters    
    for i in range(len(GenVariableNames)):
    
        extraction_def = []
        extraction_def.append(object_names[i])
        extraction_def.append(OrcFxVariableNames[i])
        extraction_def.append(objectExtras[i])
        
        extraction = ExtractTHData(model, extraction_def)
        
        df[i][cols[0]].loc[idx-1] = sim_file
        df[i][cols[1]].loc[idx-1] = extraction.getTHavg()
        df[i][cols[2]].loc[idx-1] = extraction.getTHmax()
        df[i][cols[3]].loc[idx-1] = extraction.getTHmin()
        df[i][cols[4]].loc[idx-1] = extraction.getTHabsmax()
        df[i][cols[5]].loc[idx-1] = extraction.getTHstddev()



FileNotFoundError: [WinError 3] The system cannot find the path specified: 'D:/100 - Load case batch runs/02 - Batch Runs/07 - Steady state power curve fixed floater - UMain contr WE_mode2'

In [ ]:
print(df_str)

for i in range(len(GenVariableNames)):
    df[i].to_csv (folder + '\extr_' + df_str[i]  + '.csv', header=True)



In [ ]:
# Define extractions and dataframe 
# General details
t_start_res = 0 # [s]
t_end_res = 600 # [s]

# Folder with files
folder = "D:/100 - Load case batch runs/02 - Batch Runs/11 - TSR tracking PI UMaine - Steady state"

# Make list of *.sim files
sim_files = []

os.chdir(folder)
for file in glob.glob("*.sim"):
    sim_files.append(file)

# Blade root moments and tower moments
object_names = ["15MW RWT","15MW RWT","tower","tower"]
OrcFxVariableNames = ["Root connection Ey moment", "Root connection Ex moment","x bend moment", "y bend moment"]
GenVariableNames = ["Blade Root Moment, Flapwise", "Blade Root Moment, Edgewise","Tower Base Moment, Side-Side", "Tower Base Moment, Fore-Aft"]
objectExtras = [OrcFxAPI.oeTurbine(1), OrcFxAPI.oeTurbine(1), OrcFxAPI.oeEndB, OrcFxAPI.oeEndB]

# Define dataframe for extracted values
base_df = pd.DataFrame(index=np.arange(len(sim_files)) ,columns=cols)


# define n dataframe corresponding to n variables
df_str = []
for i in range(len(GenVariableNames)):
    print('folder_11[' + GenVariableNames[i] + ']')
    df_str.append('folder_11[' + GenVariableNames[i] + ']')

df = df_str.copy()
for i in range(len(GenVariableNames)):
    df[i] = base_df.copy()
    
    
#for sim_file in sim_files:
for (idx, sim_file) in enumerate(sim_files, 1):
    
    print(sim_file)
    
    model = OrcFxAPI.Model(sim_file)
    
    # Loop through parameters    
    for i in range(len(GenVariableNames)):
    
        extraction_def = []
        extraction_def.append(object_names[i])
        extraction_def.append(OrcFxVariableNames[i])
        extraction_def.append(objectExtras[i])
        
        extraction = ExtractTHData(model, extraction_def)
        
        df[i][cols[0]].loc[idx-1] = sim_file
        df[i][cols[1]].loc[idx-1] = extraction.getTHavg()
        df[i][cols[2]].loc[idx-1] = extraction.getTHmax()
        df[i][cols[3]].loc[idx-1] = extraction.getTHmin()
        df[i][cols[4]].loc[idx-1] = extraction.getTHabsmax()
        df[i][cols[5]].loc[idx-1] = extraction.getTHstddev()



In [ ]:
print(df_str)

for i in range(len(GenVariableNames)):
    df[i].to_csv (folder + '\extr_' + df_str[i]  + '.csv', header=True)